## Part 4: Fraud

In [1]:
%run "../scripts/outlier.py" "../scripts/paths.json"

22/10/05 01:05:27 WARN Utils: Your hostname, mast30034 resolves to a loopback address: 127.0.1.1; using 45.113.234.45 instead (on interface eth0)
22/10/05 01:05:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/05 01:05:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/05 01:05:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/05 01:05:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/05 01:05:29 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


22/10/05 01:06:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# #ETL script not working, so loading the data manually for the time being: 

# #Loading consumer fraud data: 
# fraud_consumer = pd.read_csv('../data/tables/consumer_fraud_probability.csv')
# fraud_consumer = spark.createDataFrame(fraud_consumer) 

# #Loading merchant fraud data: 
# fraud_merchant = pd.read_csv('../data/tables/merchant_fraud_probabilty.csv')
# fraud_merchant = spark.createDataFrame(fraud_merchant) 


FileNotFoundError: [Errno 2] No such file or directory: '../data/tables/merchant_fraud_probabilty.csv'

The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [ ]:
# print(fraud_consumer.count())
# fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
# fraud_consumer.limit(5)

Probability of fraud for high revenue merchants...?

In [ ]:
# print(fraud_merchants.count())
# fraud_merchants = fraud_merchants.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
# fraud_merchants.limit(5)

In [ ]:

# merchant_data = final_join.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")

- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [ ]:
# #Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

# #Joining consumer fraud and merchant data together

# merchant_data.createOrReplaceTempView("temp")

# fraud_consumer.createOrReplaceTempView("temp2")

# join_1 = spark.sql(""" 

# SELECT *
# FROM temp

# LEFT JOIN temp2

# ON temp.user_id = temp2.user_id_1 
# """)


In [ ]:
# #Joining join_1 to merchant fraud 

# #Joining all test + predicting Sets Together 
# join_1.createOrReplaceTempView("temp")

# fraud_merchants.createOrReplaceTempView("temp2")

# final_join = spark.sql(""" 

# SELECT *
# FROM temp


# LEFT JOIN temp2

# ON temp.merchant_abn = temp2.merchant_abn_1
# """)

In [ ]:
# #Changing all null probability values to 0.01: 

# MIN_P = 0.01
# final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [ ]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [3]:
# Reading in segments 
segments_data = spark.read.csv('../data/curated/tagged_merchants.csv')

In [4]:
segments_data.limit(5)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
null,name,tags,merchant_abn,cleaned_tags,store_type,category
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY


In [5]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY
4,Enim Condimentum PC,([music shops - m...,10192359162,music shops musi...,3,"Books, Stationary..."
5,Fusce Company,"[(gift, card, nov...",10206519221,gift card novelty...,4,Electronics
6,Aliquam Enim Inco...,"[(computers, comP...",10255988167,computers compute...,4,Electronics
7,Ipsum Primis Ltd,"[[watch, clock, a...",10264435225,watch clock and j...,1,Toys and DIY
8,Pede Ultrices Ind...,([computer progra...,10279061213,computer programm...,2,"Beauty, Health, P..."
9,Nunc Inc.,"[(furniture, home...",10323485998,furniture home fu...,0,Furniture


In [8]:
final_dataset.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_dataset = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

22/10/05 01:07:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
final_join = final_dataset

In [16]:
#Creating a table of average fraud probabilities for each category of purchase: 
from pyspark.sql.functions import *
aggregated_prob = final_join.groupBy("temp2._c6").agg(avg("cons_fraud_prob",'merch_fraud_prob').alias("avg_consumer_fraud, 'avg_merchant_fraud')
                                                      
                                                      

AnalysisException: Column 'cons_fraud_prob' does not exist. Did you mean one of the following? [temp.address, temp.gender, temp.order_id, temp.state_code, temp.state_name, temp.take_rate, temp.SA2_code, temp.SA2_name, temp2._c0, temp2._c1, temp2._c2, temp2._c3, temp2._c4, temp2._c5, temp2._c6, temp.categories, temp.int_sa2, temp.name, temp.state, temp.total_persons, temp.user_id, temp.consumer_id, temp.merchant_abn, temp.merchant_name, temp.postcodes, temp.total_males, temp.dollar_value, temp.order_datetime, temp.revenue_levels, temp.total_females, temp.population_2020, temp.population_2021, temp.trans_merchant_abn, temp.income_2018-2019];
'Aggregate [_c6#690], [_c6#690, avg('cons_fraud_prob) AS avg(cons_fraud_prob)#1101]
+- Project [merchant_name#193, merchant_abn#31L, categories#170, take_rate#175, revenue_levels#181, name#17, address#18, state#19, gender#21, trans_merchant_abn#214L, dollar_value#41, order_id#42, order_datetime#43, user_id#35L, consumer_id#36L, postcodes#315, int_sa2

In [ ]:
aggregated_prob.show()

In [ ]:
aggregated_prob.limit(6)

In [ ]:
final_join.limit(5)

In [ ]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)



In [ ]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)

In [ ]:
model_with_fraud.limit(5)

In [ ]:
model_with_fraud.write.parquet("../data/curated/model_data_with_fraud_col.parquet") 

In [ ]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("merchant_abn_1").agg(avg("is_fraud").alias("average fraud rate per merchant"))

In [ ]:
final_model_with_fraud.limit(20)